In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [2]:
busstops = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/busstops_norway.csv')
grunnkrets_age = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_age_distribution.csv')
grunnkrets_household = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_income_households.csv')
grunnkrets_norway = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/plaace_hierarchy.csv')
sample_submission = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/sample_submission.csv')
stores_extra = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_extra.csv')
stores_test = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_test.csv')
stores_train = pd.read_csv('C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/raw/stores_train.csv')
simens_df = pd.read_csv("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/notebooks/simen/simens_dataframe-1.csv")

In [38]:
def impude_NaN (stores_df, grunnkrets_df, NaN_string ): 
    geo_df = grunnkrets_df[grunnkrets_df["year"] == 2016]
    geo_df2 = geo_df.drop("year", axis = 1)
    merged_df = stores_df.merge(geo_df2, how = "left", on = "grunnkrets_id")
    NaN_df = merged_df[merged_df[NaN_string].isna()]
    split_df = merged_df[merged_df[NaN_string].notna()]

    mat = cdist(NaN_df[['lat', 'lon']],
                split_df[['lat', 'lon']], metric='euclidean')

    new_df = pd.DataFrame(mat, index= NaN_df['grunnkrets_id'], columns=split_df['grunnkrets_id'])

    grunnkrets_id = NaN_df.grunnkrets_id
    closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)

    closest_df_with_distance = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values, "distance": distance.values})
    closest_df = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values})

    df_with_values_from_valid_id = split_df[split_df["grunnkrets_id"].isin(closest.values)]
    df_with_values_from_valid_id_removed_duplicates = df_with_values_from_valid_id.drop_duplicates(subset = ["grunnkrets_id"])

    df_valid_geo_data = df_with_values_from_valid_id_removed_duplicates.iloc[:,12:]
    df_with_only_gk_id = df_with_values_from_valid_id_removed_duplicates[["grunnkrets_id"]]
    df_list = [df_with_only_gk_id,df_valid_geo_data ]
    df_valid_geo_data_and_id = pd.concat(df_list, axis=1)

    df_without_nan = NaN_df.iloc[:,:12]

    df_including_closest_valid_id = df_without_nan.reset_index().merge(closest_df, how = "left", on = "grunnkrets_id").set_index("index")

    df_impuded = df_including_closest_valid_id.reset_index().merge(df_valid_geo_data_and_id, how="left", left_on ="closest_valid_id", right_on="grunnkrets_id").set_index("index")
    df_impuded_without_duplicates = df_impuded.drop_duplicates(subset="store_id")

    new_df_impuded = df_impuded_without_duplicates.drop(["closest_valid_id", "grunnkrets_id_y"], axis = 1).rename(columns= {"grunnkrets_id_x": "grunnkrets_id"})


    impuded = pd.concat([split_df, new_df_impuded])

    return impuded




In [39]:
def impude_NaN_test_set(test_df, grunnkrets_df, NaN_string): 
    geo_df = grunnkrets_df[grunnkrets_df["year"] == 2016]
    geo_df2 = geo_df.drop("year", axis = 1)
    merged_df = test_df.merge(geo_df2, how = "left", on = "grunnkrets_id")
    NaN_df = merged_df[merged_df[NaN_string].isna()]
    split_df = merged_df[merged_df[NaN_string].notna()]

    mat = cdist(NaN_df[['lat', 'lon']],
                split_df[['lat', 'lon']], metric='euclidean')

    new_df = pd.DataFrame(mat, index= NaN_df['grunnkrets_id'], columns=split_df['grunnkrets_id'])

    grunnkrets_id = NaN_df.grunnkrets_id
    closest = new_df.idxmin(axis=1)
    distance = new_df.min(axis=1)

    closest_df_with_distance = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values, "distance": distance.values})
    closest_df = pd.DataFrame({"grunnkrets_id": grunnkrets_id, "closest_valid_id" : closest.values})

    df_with_values_from_valid_id = split_df[split_df["grunnkrets_id"].isin(closest.values)]
    df_with_values_from_valid_id_removed_duplicates = df_with_values_from_valid_id.drop_duplicates(subset = ["grunnkrets_id"])

    df_valid_geo_data = df_with_values_from_valid_id_removed_duplicates.iloc[:,11:]
    df_with_only_gk_id = df_with_values_from_valid_id_removed_duplicates[["grunnkrets_id"]]
    df_list = [df_with_only_gk_id,df_valid_geo_data ]
    df_valid_geo_data_and_id = pd.concat(df_list, axis=1)

    df_without_nan = NaN_df.iloc[:,:11]

    df_including_closest_valid_id = df_without_nan.reset_index().merge(closest_df, how = "left", on = "grunnkrets_id").set_index("index")

    df_impuded = df_including_closest_valid_id.reset_index().merge(df_valid_geo_data_and_id, how="left", left_on ="closest_valid_id", right_on="grunnkrets_id").set_index("index")
    df_impuded_without_duplicates = df_impuded.drop_duplicates(subset="store_id")

    new_df_impuded = df_impuded_without_duplicates.drop(["closest_valid_id", "grunnkrets_id_y"], axis = 1).rename(columns= {"grunnkrets_id_x": "grunnkrets_id"})


    impuded = pd.concat([split_df, new_df_impuded])

    return impuded

In [40]:
stores_gk_impuded = impude_NaN(stores_train, grunnkrets_norway, NaN_string="district_name")
stores_house_impuded = impude_NaN(stores_train, grunnkrets_household, "singles")
stores_age_impuded = impude_NaN(stores_train, grunnkrets_age, NaN_string="age_0")
stores_test_impuded = impude_NaN_test_set(stores_test, grunnkrets_norway, NaN_string="district_name")




In [49]:
def population_age_impuded(age_impuded_df):
    age_part = age_impuded_df.iloc[:,12:]
    age_gk_id = age_impuded_df[["grunnkrets_id"]]
    age_df = pd.concat([age_gk_id, age_part], axis = 1)
    population = age_df.drop(["grunnkrets_id"], axis=1).sum(axis=1)
    age_df["population_count"] = population
    return age_df[["grunnkrets_id", "population_count"]]

df = population_age_impuded(stores_age_impuded)

In [62]:

def population_grouped_impuded(age_impuded_df, grunnkrets_df, grouping_element):
    age_df = population_age_impuded(age_impuded_df)
    geo_df = grunnkrets_df[grunnkrets_df["year"] == 2016]
    population_df = age_df.merge(geo_df, how="left", on="grunnkrets_id")
    grouped_df = population_df.groupby([grouping_element], as_index=False)[
        "population_count"].sum()
    return grouped_df

df = population_grouped_impuded(stores_age_impuded, grunnkrets_norway, grouping_element = "district_name")
df

,district_name,population_count
0,Agdenes,297.0
1,Alfaset,22342.0
2,Alstad,2974.0
3,Alstahaug/Tjøtta,185.0
4,Alta sentrum nord,5975.0
...,...,...
1250,Øymark,2122.0
1251,Øyrekken,914.0
1252,Øystese,8687.0
1253,Øystre Slidre nord,8017.0


In [61]:
age = population_age_impuded(stores_age_impuded)
geo = grunnkrets_norway[grunnkrets_norway["year"]==2016]
combined = age.merge(stores_gk_impuded, how = "left", on = "grunnkrets_id")
combined2 = combined.groupby(["district_name"], as_index = False)["population_count"].sum()
combined2

,district_name,population_count
0,Agdenes,297.0
1,Alfaset,72022.0
2,Alstad,5430.0
3,Alstahaug/Tjøtta,185.0
4,Alta sentrum nord,14255.0
...,...,...
1250,Øymark,6068.0
1251,Øyrekken,914.0
1252,Øystese,76295.0
1253,Øystre Slidre nord,96901.0


In [3]:
train_df = pd.read_csv("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/train_data.csv")
test_df = pd.read_csv("C:/Users/aminp/OneDrive/Dokumenter/NTNU/4år/Machinelearning/machine_learning/data/train_data/test_data.csv")



In [4]:
train_df

,Unnamed: 0,store_id,revenue,is_mall,is_chain,aggtransformer-1__store_count,aggtransformer-2__store_count,aggtransformer-3__store_count,aggtransformer-4__store_count,aggtransformer-5__store_count,...,remainder__number_of_lokalt_knutepunkt,remainder__number_of_nasjonalt_knutepunkt,remainder__number_of_regionalt_knutepunkt,remainder__number_of_annen_viktig_holdeplass,remainder__distance_to_mangler_viktighetsnivå,remainder__distance_to_standard_holdeplass,remainder__distance_to_lokalt_knutepunkt,remainder__distance_to_nasjonalt_knutepunkt,remainder__distance_to_regionalt_knutepunkt,remainder__distance_to_annen_viktig_holdeplass
0,0,983540538-974187930-44774,17.998,True,True,28.0,19.0,1.0,78.0,58.0,...,25,0,1,0,0.005315,0.012600,0.005467,0.294614,0.016952,0.155313
1,1,987074191-973117734-44755,23.828,False,True,129.0,91.0,3.0,1220.0,856.0,...,74,10,13,0,0.001579,0.045599,0.001257,0.020375,0.001559,0.116319
2,2,984890265-981157303-64491,16.099,True,True,25.0,19.0,1.0,28.0,21.0,...,12,0,5,7,0.000371,0.011161,0.025783,0.326362,0.017330,0.054763
3,3,914057442-992924179-126912,9.296,True,True,240.0,174.0,7.0,62.0,46.0,...,21,2,9,0,0.005329,0.010858,0.016854,0.012352,0.000522,5.257526
4,4,913018583-913063538-668469,4.528,True,True,18.0,10.0,2.0,252.0,170.0,...,25,0,0,6,0.003096,0.002093,0.003370,0.319223,0.644539,0.006737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12854,12854,915789943-915806929-781991,0.088,False,False,785.0,143.0,11.0,92.0,21.0,...,6,0,4,3,0.001349,0.008576,0.006203,0.176835,0.005978,0.009062
12855,12855,917921733-917982368-868081,1.816,True,True,785.0,143.0,11.0,149.0,28.0,...,18,0,2,0,0.002951,0.118968,0.022918,0.333546,0.051408,0.168095
12856,12856,911721961-911764474-496764,38.225,True,True,157.0,12.0,2.0,324.0,41.0,...,13,0,15,24,0.001587,0.102015,0.025107,0.249433,0.032679,0.037453
12857,12857,914337046-914343372-721294,3.642,True,False,785.0,143.0,11.0,343.0,57.0,...,4,0,0,0,0.001097,0.005468,0.080072,0.116255,0.144443,0.101411


In [6]:
test_df.isna()

Unnamed: 0                                           0
store_id                                             0
is_mall                                              0
is_chain                                             0
aggtransformer-1__store_count                      271
                                                  ... 
remainder__distance_to_standard_holdeplass        8577
remainder__distance_to_lokalt_knutepunkt          8577
remainder__distance_to_nasjonalt_knutepunkt       8577
remainder__distance_to_regionalt_knutepunkt       8577
remainder__distance_to_annen_viktig_holdeplass    8577
Length: 168, dtype: int64